In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [89]:
CLIENT_ID = 'RM24NDAJSW52KGHMEUPAOMKEE1VVLBNO1OCV4SUJ4EKVXFNS' 
# your Foursquare ID
CLIENT_SECRET = 'KVR4B32R0H3JGH31BCCPK30REKHYPMSKJXM1ILNAKFDWVTXN' 
# your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RM24NDAJSW52KGHMEUPAOMKEE1VVLBNO1OCV4SUJ4EKVXFNS
CLIENT_SECRET:KVR4B32R0H3JGH31BCCPK30REKHYPMSKJXM1ILNAKFDWVTXN


In [3]:
website_url = requests.get('https://en.wikipedia.org/wiki/NW_postcode_area').text

from bs4 import BeautifulSoup
# extract the whole website
soup = BeautifulSoup(website_url,'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   NW postcode area - Wikipedia
  </title>
  <script>
   document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"NW_postcode_area","wgTitle":"NW postcode area","wgCurRevisionId":898418628,"wgRevisionId":898418628,"wgArticleId":7417890,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Articles using KML from Wikidata","Use dmy dates from October 2016","London articles missing geocoordinate data","All articles needing coordinates","Postcode areas covering London","Media and communications in the London Borough of Brent","Media and communications in the London Borough of Barnet","Media and communications in the London Borough of

In [4]:
# extract the whole table
My_table = soup.find('table',{'class':'wikitable sortable'})

In [5]:
print(My_table)

<table class="wikitable sortable" style="font-size:95%">
<tbody><tr style="white-space:nowrap">
<th>Postcode district
</th>
<th><a href="/wiki/Post_town" title="Post town">Post town</a>
</th>
<th>Coverage
</th>
<th>Local authority area
</th></tr>
<tr>
<th>NW1
</th>
<td>LONDON
</td>
<td><i>North Western head district:</i> <a href="/wiki/Marylebone" title="Marylebone">Marylebone</a> (part), <a class="mw-redirect" href="/wiki/Euston,_London" title="Euston, London">Euston</a>, <a href="/wiki/Regent%27s_Park" title="Regent's Park">Regent's Park</a>, <a href="/wiki/Baker_Street" title="Baker Street">Baker Street</a>, <a href="/wiki/Camden_Town" title="Camden Town">Camden Town</a>,  <a href="/wiki/Somers_Town,_London" title="Somers Town, London">Somers Town</a>, <a href="/wiki/Primrose_Hill" title="Primrose Hill">Primrose Hill</a> (part) and <a href="/wiki/Lisson_Grove" title="Lisson Grove">Lisson Grove</a> (part)
</td>
<td><a href="/wiki/London_Borough_of_Camden" title="London Borough of Cam

#### extract postal code NW in London

In [16]:
lists = My_table.findAll('th')
NWlist = []
for item in lists:
    if str(item).find('NW') != -1:
        if str(item).find('#FFFFF') == -1:
            tmpstr = str(item).replace('<th>','').replace('\n</th>','')
            NWlist.append(tmpstr)
print(NWlist)

['NW1', 'NW2', 'NW3', 'NW4', 'NW5', 'NW6', 'NW7', 'NW8', 'NW9', 'NW10', 'NW11']


In [26]:
len(NWlist)

11

#### extract other useful info

In [28]:
lists = My_table.findAll('tr')
list2 = str(lists).split('<tr>')
list3 = []
for item in list2:
    if item.find('#FFFFF') == -1:
        list3.append(item)
list3 = list3[1:]
        
print(len(list3))

11


In [52]:
list3[1].split('<td>')

['\n<th>NW2\n</th>\n',
 'LONDON\n</td>\n',
 '<i>Cricklewood district:</i> <a href="/wiki/Cricklewood" title="Cricklewood">Cricklewood</a>, <a href="/wiki/Dollis_Hill" title="Dollis Hill">Dollis Hill</a>, <a href="/wiki/Childs_Hill" title="Childs Hill">Childs Hill</a>, <a href="/wiki/Golders_Green" title="Golders Green">Golders Green</a> (part), <a href="/wiki/Brent_Cross" title="Brent Cross">Brent Cross</a> (part), <a href="/wiki/Willesden" title="Willesden">Willesden</a> (north), <a href="/wiki/Neasden" title="Neasden">Neasden</a> (north)\n</td>\n',
 '<a href="/wiki/London_Borough_of_Barnet" title="London Borough of Barnet">Barnet</a>, <a href="/wiki/London_Borough_of_Brent" title="London Borough of Brent">Brent</a>, Camden\n</td></tr>, ']

In [73]:
Borlist = []
Neilist = []
for i in range(len(list3)):
    tmp2 = list3[i].split('<td>')
    for item in range(len(tmp2)):
        if item in range(2):
            pass
        elif item == len(tmp2)-1:
            tmp3 = tmp2[item].split(',')
            tmpb = []
            for item2 in tmp3:
                if item2.find('<a') != -1:
                    tmpb.append(item2[item2.find('">')+2:item2.find('</a>')])
                else:
                    tmpb.append(item2.replace(' ','').replace('\n','').replace('</td></tr>',''))            
            Borlist.append(tmpb[:-1])
        else:
            tmp3 = tmp2[item].split('<a')
            tmpn = []
            for item2 in tmp3:
                if item2.find('</a>') != -1:
                    tmpn.append(item2[item2.find('">')+2:item2.find('</a>')])
            Neilist.append(tmpn)
        
        
        

In [74]:
print(Borlist)
print(Neilist)

[['Camden', 'Westminster'], ['Barnet', 'Brent', 'Camden'], ['Camden', 'Barnet'], ['Barnet'], ['Camden', 'Islington'], ['Brent', 'Camden', 'Westminster'], ['Barnet'], ['Westminster', 'Camden'], ['Barnet', 'Brent', 'Harrow'], ['Brent', 'Ealing', 'Hammersmith and Fulham', 'Kensington and Chelsea'], ['Barnet']]
[['Marylebone', 'Euston', "Regent's Park", 'Baker Street', 'Camden Town', 'Somers Town', 'Primrose Hill', 'Lisson Grove'], ['Cricklewood', 'Dollis Hill', 'Childs Hill', 'Golders Green', 'Brent Cross', 'Willesden', 'Neasden'], ['Hampstead', 'Belsize Park', 'Frognal', 'Childs Hill', 'South Hampstead', 'Swiss Cottage', 'Primrose Hill', 'Chalk Farm', 'Gospel Oak'], ['Hendon', 'Brent Cross'], ['Kentish Town', 'Camden Town', 'Gospel Oak', 'Dartmouth Park', 'Chalk Farm', 'Tufnell Park'], ['Kilburn', 'Brondesbury', 'West Hampstead', "Queen's Park", 'Kensal Green', 'South Hampstead', 'Swiss Cottage'], ['Mill Hill', 'Arkley', 'Edgware'], ["St John's Wood", 'Primrose Hill', 'Marylebone', 'Liss

In [76]:
len(Neilist)

11

#### Create dataframe

In [79]:
PC1,BO1,NE1 = [],[],[]
for i in range(len(NWlist)):
    lenB = len(Borlist[i])
    for j in range(lenB):
        PC1.append(NWlist[i])
        NE1.append(','.join(Neilist[i]))
        BO1.append(Borlist[i][j])
print(len(PC1))
print(len(BO1))
print(len(NE1))

24
24
24


In [80]:
df = pd.DataFrame()
df['PostalCode'] = PC1
df['Borough'] = BO1
df['Neighborhood'] = NE1

In [81]:
df

,PostalCode,Borough,Neighborhood
0,NW1,Camden,"Marylebone,Euston,Regent's Park,Baker Street,C..."
1,NW1,Westminster,"Marylebone,Euston,Regent's Park,Baker Street,C..."
2,NW2,Barnet,"Cricklewood,Dollis Hill,Childs Hill,Golders Gr..."
3,NW2,Brent,"Cricklewood,Dollis Hill,Childs Hill,Golders Gr..."
4,NW2,Camden,"Cricklewood,Dollis Hill,Childs Hill,Golders Gr..."
5,NW3,Camden,"Hampstead,Belsize Park,Frognal,Childs Hill,Sou..."
6,NW3,Barnet,"Hampstead,Belsize Park,Frognal,Childs Hill,Sou..."
7,NW4,Barnet,"Hendon,Brent Cross"
8,NW5,Camden,"Kentish Town,Camden Town,Gospel Oak,Dartmouth ..."
9,NW5,Islington,"Kentish Town,Camden Town,Gospel Oak,Dartmouth ..."


In [84]:
Bor = ['Camden','Westminster','Barnet','Brent','Islington','Harrow','Ealing','Hammersmith and Fulham','Kensington and Chelsea']
Lat = [51.5333312,51.510357,51.65,51.559664428,51.5362206,51.583015,51.499998,51.48,51.4951]
Lon = [-0.166666,-0.116773,-0.2,-0.270998916,-0.10304,-0.337820,-0.333332,-0.21,-0.2061]
print(len(Bor))
print(len(Lat))
print(len(Lon))
geoB = pd.DataFrame()
geoB['Borough'] = Bor
geoB['Latitude'] = Lat
geoB['Longitude'] = Lon

9
9
9


In [86]:
geoB

,Borough,Latitude,Longitude
0,Camden,51.533331,-0.166666
1,Westminster,51.510357,-0.116773
2,Barnet,51.650000,-0.200000
3,Brent,51.559664,-0.270999
4,Islington,51.536221,-0.103040
5,Harrow,51.583015,-0.337820
6,Ealing,51.499998,-0.333332
7,Hammersmith and Fulham,51.480000,-0.210000
8,Kensington and Chelsea,51.495100,-0.206100


In [87]:
df2 = pd.merge(df,geoB)

In [88]:
df2

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,NW1,Camden,"Marylebone,Euston,Regent's Park,Baker Street,C...",51.533331,-0.166666
1,NW2,Camden,"Cricklewood,Dollis Hill,Childs Hill,Golders Gr...",51.533331,-0.166666
2,NW3,Camden,"Hampstead,Belsize Park,Frognal,Childs Hill,Sou...",51.533331,-0.166666
3,NW5,Camden,"Kentish Town,Camden Town,Gospel Oak,Dartmouth ...",51.533331,-0.166666
4,NW6,Camden,"Kilburn,Brondesbury,West Hampstead,Queen's Par...",51.533331,-0.166666
5,NW8,Camden,"St John's Wood,Primrose Hill,Marylebone,Lisson...",51.533331,-0.166666
6,NW1,Westminster,"Marylebone,Euston,Regent's Park,Baker Street,C...",51.510357,-0.116773
7,NW6,Westminster,"Kilburn,Brondesbury,West Hampstead,Queen's Par...",51.510357,-0.116773
8,NW8,Westminster,"St John's Wood,Primrose Hill,Marylebone,Lisson...",51.510357,-0.116773
9,NW2,Barnet,"Cricklewood,Dollis Hill,Childs Hill,Golders Gr...",51.650000,-0.200000


In [92]:
from operator import itemgetter
from itertools import chain
nei = df2.columns.difference(['Neighborhood'])
genres = df2['Neighborhood'].str.split(',')

df3 =  (df2.loc[df2.index.repeat(genres.str.len()),nei]
         .assign(Neighborhood=list(chain.from_iterable(genres.tolist())))).reset_index(drop=True)


In [94]:
df3.head()

,Borough,Latitude,Longitude,PostalCode,Neighborhood
0,Camden,51.533331,-0.166666,NW1,Marylebone
1,Camden,51.533331,-0.166666,NW1,Euston
2,Camden,51.533331,-0.166666,NW1,Regent's Park
3,Camden,51.533331,-0.166666,NW1,Baker Street
4,Camden,51.533331,-0.166666,NW1,Camden Town


#### Only with Camden

In [139]:
address = 'London'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


In [153]:
camden_data = df3#[df3['Borough'] == 'Westminster'].reset_index(drop=True)
camden_data.head()

,Borough,Latitude,Longitude,PostalCode,Neighborhood
0,Camden,51.533331,-0.166666,NW1,Marylebone
1,Camden,51.533331,-0.166666,NW1,Euston
2,Camden,51.533331,-0.166666,NW1,Regent's Park
3,Camden,51.533331,-0.166666,NW1,Baker Street
4,Camden,51.533331,-0.166666,NW1,Camden Town


In [166]:
address = 'Camden, UK'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Camden are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Camden are 51.5423045, -0.1395604.


In [167]:
# create map of Manhattan using latitude and longitude values
map_camden = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(camden_data['Latitude'], camden_data['Longitude'], camden_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_camden)  
    
map_camden

In [168]:
camden_data.loc[0, 'Neighborhood']

'Marylebone'

In [169]:
neighborhood_latitude = camden_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = camden_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = camden_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marylebone are 51.5333312, -0.166666.


In [170]:
radius = 500
LIMIT = 30
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [171]:
results = requests.get(url).json()

In [172]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [173]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Chicken Shop & Dirty Burger,Fast Food Restaurant,51.533543,-0.170232
1,The Good Life Eatery,Salad Place,51.532252,-0.169289
2,Oslo Court,French Restaurant,51.533364,-0.166508
3,Drunch Regent's Park,Restaurant,51.535029,-0.168829
4,St John's Wood Church Gardens,Garden,51.530741,-0.169392


In [174]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

27 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Camden

In [175]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [176]:
camden_venues = getNearbyVenues(names=camden_data['Neighborhood'],
                                   latitudes=camden_data['Latitude'],
                                   longitudes=camden_data['Longitude'])

Marylebone
Euston
Regent's Park
Baker Street
Camden Town
Somers Town
Primrose Hill
Lisson Grove
Cricklewood
Dollis Hill
Childs Hill
Golders Green
Brent Cross
Willesden
Neasden
Hampstead
Belsize Park
Frognal
Childs Hill
South Hampstead
Swiss Cottage
Primrose Hill
Chalk Farm
Gospel Oak
Kentish Town
Camden Town
Gospel Oak
Dartmouth Park
Chalk Farm
Tufnell Park
Kilburn
Brondesbury
West Hampstead
Queen's Park
Kensal Green
South Hampstead
Swiss Cottage
St John's Wood
Primrose Hill
Marylebone
Lisson Grove
Marylebone
Euston
Regent's Park
Baker Street
Camden Town
Somers Town
Primrose Hill
Lisson Grove
Kilburn
Brondesbury
West Hampstead
Queen's Park
Kensal Green
South Hampstead
Swiss Cottage
St John's Wood
Primrose Hill
Marylebone
Lisson Grove
Cricklewood
Dollis Hill
Childs Hill
Golders Green
Brent Cross
Willesden
Neasden
Hampstead
Belsize Park
Frognal
Childs Hill
South Hampstead
Swiss Cottage
Primrose Hill
Chalk Farm
Gospel Oak
Hendon
Brent Cross
Mill Hill
Arkley
Edgware
The Hyde
Colindale
King

In [177]:
print(camden_venues.shape)
camden_venues.head()

(3279, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marylebone,51.533331,-0.166666,Chicken Shop & Dirty Burger,51.533543,-0.170232,Fast Food Restaurant
1,Marylebone,51.533331,-0.166666,The Good Life Eatery,51.532252,-0.169289,Salad Place
2,Marylebone,51.533331,-0.166666,Oslo Court,51.533364,-0.166508,French Restaurant
3,Marylebone,51.533331,-0.166666,Drunch Regent's Park,51.535029,-0.168829,Restaurant
4,Marylebone,51.533331,-0.166666,St John's Wood Church Gardens,51.530741,-0.169392,Garden


In [178]:
camden_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Arkley,15,15,15,15,15,15
Baker Street,57,57,57,57,57,57
Belsize Park,42,42,42,42,42,42
Brent Cross,76,76,76,76,76,76
Brent Park,62,62,62,62,62,62
Brondesbury,61,61,61,61,61,61
Camden Town,114,114,114,114,114,114
Chalk Farm,99,99,99,99,99,99
Childs Hill,88,88,88,88,88,88


## Analyzing each neighborhood

In [179]:
# one hot encoding
camden_onehot = pd.get_dummies(camden_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
camden_onehot['Neighborhood'] =camden_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [camden_onehot.columns[-1]] + list(camden_onehot.columns[:-1])
camden_onehot = camden_onehot[fixed_columns]

camden_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Bakery,Bar,Beer Bar,Bookstore,Boutique,Breakfast Spot,Brewery,Burger Joint,Bus Stop,Café,Canal,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,English Restaurant,Event Space,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Food Court,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Kebab Restaurant,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Music Store,Park,Persian Restaurant,Pharmacy,Pizza Place,Portuguese Restaurant,Pub,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Soba Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Theater,Thrift / Vintage Store,Track,Turkish Restaurant,Warehouse Store,Wine Shop,Women's Store
0,Marylebone,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Marylebone,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Marylebone,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Marylebone,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Marylebone,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [180]:
camden_onehot.shape

(3279, 87)

In [181]:
camden_grouped = camden_onehot.groupby('Neighborhood').mean().reset_index()

In [182]:
num_top_venues = 5

for hood in camden_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = camden_grouped[camden_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arkley----
               venue  freq
0  Convenience Store  0.13
1           Bus Stop  0.13
2     Sandwich Place  0.07
3               Park  0.07
4     Breakfast Spot  0.07


----Baker Street----
         venue  freq
0      Theater  0.11
1        Hotel  0.05
2   Restaurant  0.05
3          Pub  0.05
4  Coffee Shop  0.05


----Belsize Park----
        venue  freq
0    Bus Stop  0.07
1         Pub  0.07
2  Restaurant  0.05
3      Bakery  0.05
4        Café  0.05


----Brent Cross----
               venue  freq
0           Bus Stop  0.09
1  Convenience Store  0.08
2                Pub  0.07
3         Restaurant  0.05
4             Bakery  0.05


----Brent Park----
                venue  freq
0                Café  0.10
1                 Pub  0.08
2               Hotel  0.06
3                Park  0.06
4  Italian Restaurant  0.06


----Brondesbury----
          venue  freq
0       Theater  0.10
1   Coffee Shop  0.05
2         Hotel  0.05
3    Restaurant  0.05
4  Cocktail Bar  0.05


--

In [183]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [184]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = camden_grouped['Neighborhood']

for ind in np.arange(camden_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(camden_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(5)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arkley,Convenience Store,Bus Stop,Breakfast Spot,Sandwich Place,Chinese Restaurant,Park,Indian Restaurant,Pizza Place,Pub,Restaurant
1,Baker Street,Theater,Cocktail Bar,Coffee Shop,Hotel,Pub,Restaurant,Deli / Bodega,Café,Japanese Restaurant,French Restaurant
2,Belsize Park,Pub,Bus Stop,Restaurant,Deli / Bodega,Convenience Store,Café,Sandwich Place,Bakery,Pharmacy,Fast Food Restaurant
3,Brent Cross,Bus Stop,Convenience Store,Pub,Restaurant,Sandwich Place,Bakery,Metro Station,Pizza Place,Breakfast Spot,Grocery Store
4,Brent Park,Café,Pub,Park,Hotel,Italian Restaurant,Persian Restaurant,Bakery,Pizza Place,Grocery Store,Restaurant


## 4. Cluster Neighborhoods

In [185]:
# set number of clusters
kclusters = 5
camden_grouped_clustering = camden_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(camden_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 2, 4, 4, 1, 2, 2, 4, 4, 0], dtype=int32)

In [186]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

camden_merged = camden_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
camden_merged = camden_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

camden_merged.head() # check the last columns!

,Borough,Latitude,Longitude,PostalCode,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Camden,51.533331,-0.166666,NW1,Marylebone,2,Theater,Cocktail Bar,Coffee Shop,Hotel,Pub,Restaurant,Deli / Bodega,Café,Japanese Restaurant,French Restaurant
1,Camden,51.533331,-0.166666,NW1,Euston,2,Theater,Cocktail Bar,Coffee Shop,Hotel,Pub,Restaurant,Deli / Bodega,Café,Japanese Restaurant,French Restaurant
2,Camden,51.533331,-0.166666,NW1,Regent's Park,2,Theater,Cocktail Bar,Coffee Shop,Hotel,Pub,Restaurant,Deli / Bodega,Café,Japanese Restaurant,French Restaurant
3,Camden,51.533331,-0.166666,NW1,Baker Street,2,Theater,Cocktail Bar,Coffee Shop,Hotel,Pub,Restaurant,Deli / Bodega,Café,Japanese Restaurant,French Restaurant
4,Camden,51.533331,-0.166666,NW1,Camden Town,2,Pub,Coffee Shop,Theater,Café,Restaurant,Hotel,Deli / Bodega,Cocktail Bar,French Restaurant,Ice Cream Shop


In [187]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(camden_merged['Latitude'], camden_merged['Longitude'], camden_merged['Neighborhood'], camden_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [188]:
camden_merged.loc[camden_merged['Cluster Labels'] == 0, camden_merged.columns[[1] + list(range(5, camden_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
81,51.650000,0,Coffee Shop,Gym,Chinese Restaurant,Pizza Place,Sandwich Place,Convenience Store,Portuguese Restaurant,Clothing Store,Bus Stop,Indian Restaurant
82,51.650000,0,Coffee Shop,Gym,Chinese Restaurant,Pizza Place,Sandwich Place,Convenience Store,Portuguese Restaurant,Clothing Store,Bus Stop,Indian Restaurant
83,51.650000,0,Coffee Shop,Gym,Chinese Restaurant,Pizza Place,Sandwich Place,Convenience Store,Portuguese Restaurant,Clothing Store,Bus Stop,Indian Restaurant
84,51.650000,0,Coffee Shop,Gym,Chinese Restaurant,Pizza Place,Sandwich Place,Convenience Store,Portuguese Restaurant,Clothing Store,Bus Stop,Indian Restaurant
85,51.650000,0,Coffee Shop,Gym,Chinese Restaurant,Pizza Place,Sandwich Place,Convenience Store,Portuguese Restaurant,Clothing Store,Bus Stop,Indian Restaurant
86,51.650000,0,Coffee Shop,Gym,Chinese Restaurant,Pizza Place,Sandwich Place,Convenience Store,Portuguese Restaurant,Clothing Store,Bus Stop,Indian Restaurant
106,51.559664,0,Coffee Shop,Gym,Chinese Restaurant,Pizza Place,Sandwich Place,Convenience Store,Portuguese Restaurant,Clothing Store,Bus Stop,Indian Restaurant
107,51.559664,0,Coffee Shop,Gym,Chinese Restaurant,Pizza Place,Sandwich Place,Convenience Store,Portuguese Restaurant,Clothing Store,Bus Stop,Indian Restaurant
108,51.559664,0,Coffee Shop,Gym,Chinese Restaurant,Pizza Place,Sandwich Place,Convenience Store,Portuguese Restaurant,Clothing Store,Bus Stop,Indian Restaurant
109,51.559664,0,Coffee Shop,Gym,Chinese Restaurant,Pizza Place,Sandwich Place,Convenience Store,Portuguese Restaurant,Clothing Store,Bus Stop,Indian Restaurant


In [189]:
camden_merged.loc[camden_merged['Cluster Labels'] == 1, camden_merged.columns[[1] + list(range(5, camden_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,51.533331,1,Café,Pub,Park,Hotel,Italian Restaurant,Warehouse Store,Bus Stop,Bakery,Restaurant,Sandwich Place
14,51.533331,1,Café,Pub,Park,Hotel,Italian Restaurant,Warehouse Store,Bus Stop,Bakery,Restaurant,Sandwich Place
34,51.533331,1,Pub,Café,Hotel,Theater,Restaurant,Italian Restaurant,Park,Warehouse Store,Cocktail Bar,Persian Restaurant
53,51.510357,1,Pub,Café,Hotel,Theater,Restaurant,Italian Restaurant,Park,Warehouse Store,Cocktail Bar,Persian Restaurant
65,51.650000,1,Café,Pub,Park,Hotel,Italian Restaurant,Warehouse Store,Bus Stop,Bakery,Restaurant,Sandwich Place
66,51.650000,1,Café,Pub,Park,Hotel,Italian Restaurant,Warehouse Store,Bus Stop,Bakery,Restaurant,Sandwich Place
97,51.559664,1,Café,Pub,Park,Hotel,Italian Restaurant,Warehouse Store,Bus Stop,Bakery,Restaurant,Sandwich Place
98,51.559664,1,Café,Pub,Park,Hotel,Italian Restaurant,Warehouse Store,Bus Stop,Bakery,Restaurant,Sandwich Place
103,51.559664,1,Pub,Café,Hotel,Theater,Restaurant,Italian Restaurant,Park,Warehouse Store,Cocktail Bar,Persian Restaurant
112,51.559664,1,Café,Pub,Park,Hotel,Italian Restaurant,Warehouse Store,Bus Stop,Bakery,Restaurant,Sandwich Place


In [190]:
camden_merged.loc[camden_merged['Cluster Labels'] == 2, camden_merged.columns[[1] + list(range(5, camden_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,51.533331,2,Theater,Cocktail Bar,Coffee Shop,Hotel,Pub,Restaurant,Deli / Bodega,Café,Japanese Restaurant,French Restaurant
1,51.533331,2,Theater,Cocktail Bar,Coffee Shop,Hotel,Pub,Restaurant,Deli / Bodega,Café,Japanese Restaurant,French Restaurant
2,51.533331,2,Theater,Cocktail Bar,Coffee Shop,Hotel,Pub,Restaurant,Deli / Bodega,Café,Japanese Restaurant,French Restaurant
3,51.533331,2,Theater,Cocktail Bar,Coffee Shop,Hotel,Pub,Restaurant,Deli / Bodega,Café,Japanese Restaurant,French Restaurant
4,51.533331,2,Pub,Coffee Shop,Theater,Café,Restaurant,Hotel,Deli / Bodega,Cocktail Bar,French Restaurant,Ice Cream Shop
5,51.533331,2,Theater,Cocktail Bar,Coffee Shop,Hotel,Pub,Restaurant,Deli / Bodega,Café,Japanese Restaurant,French Restaurant
6,51.533331,2,Theater,Pub,Restaurant,Coffee Shop,Hotel,Cocktail Bar,Deli / Bodega,Café,Bus Stop,Sandwich Place
7,51.533331,2,Theater,Cocktail Bar,Coffee Shop,Hotel,Pub,Restaurant,Deli / Bodega,Café,Japanese Restaurant,French Restaurant
19,51.533331,2,Theater,Pub,Restaurant,Coffee Shop,Café,Deli / Bodega,Bus Stop,Hotel,Cocktail Bar,Bakery
20,51.533331,2,Theater,Pub,Restaurant,Coffee Shop,Café,Deli / Bodega,Bus Stop,Hotel,Cocktail Bar,Bakery


In [191]:
camden_merged.loc[camden_merged['Cluster Labels'] == 3, camden_merged.columns[[1] + list(range(5, camden_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
76,51.65,3,Convenience Store,Bus Stop,Breakfast Spot,Sandwich Place,Chinese Restaurant,Park,Indian Restaurant,Pizza Place,Pub,Restaurant
78,51.65,3,Convenience Store,Bus Stop,Breakfast Spot,Sandwich Place,Chinese Restaurant,Park,Indian Restaurant,Pizza Place,Pub,Restaurant
79,51.65,3,Convenience Store,Bus Stop,Breakfast Spot,Sandwich Place,Chinese Restaurant,Park,Indian Restaurant,Pizza Place,Pub,Restaurant
80,51.65,3,Convenience Store,Bus Stop,Breakfast Spot,Sandwich Place,Chinese Restaurant,Park,Indian Restaurant,Pizza Place,Pub,Restaurant
88,51.65,3,Convenience Store,Bus Stop,Breakfast Spot,Sandwich Place,Chinese Restaurant,Park,Indian Restaurant,Pizza Place,Pub,Restaurant
89,51.65,3,Convenience Store,Bus Stop,Breakfast Spot,Sandwich Place,Chinese Restaurant,Park,Indian Restaurant,Pizza Place,Pub,Restaurant
90,51.65,3,Convenience Store,Bus Stop,Breakfast Spot,Sandwich Place,Chinese Restaurant,Park,Indian Restaurant,Pizza Place,Pub,Restaurant


In [192]:
camden_merged.loc[camden_merged['Cluster Labels'] == 4, camden_merged.columns[[1] + list(range(5, camden_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,51.533331,4,Pub,Bus Stop,Convenience Store,Café,Restaurant,Sandwich Place,Bakery,Deli / Bodega,Warehouse Store,Coffee Shop
9,51.533331,4,Pub,Bus Stop,Convenience Store,Café,Restaurant,Sandwich Place,Bakery,Deli / Bodega,Warehouse Store,Coffee Shop
10,51.533331,4,Pub,Bus Stop,Bakery,Restaurant,Café,Convenience Store,Deli / Bodega,Sandwich Place,Metro Station,Garden
11,51.533331,4,Bus Stop,Convenience Store,Pub,Restaurant,Sandwich Place,Bakery,Metro Station,Deli / Bodega,Pizza Place,Breakfast Spot
12,51.533331,4,Bus Stop,Convenience Store,Pub,Restaurant,Sandwich Place,Bakery,Metro Station,Pizza Place,Breakfast Spot,Grocery Store
15,51.533331,4,Pub,Bus Stop,Restaurant,Deli / Bodega,Convenience Store,Café,Sandwich Place,Bakery,Pharmacy,Fast Food Restaurant
16,51.533331,4,Pub,Bus Stop,Restaurant,Deli / Bodega,Convenience Store,Café,Sandwich Place,Bakery,Pharmacy,Fast Food Restaurant
17,51.533331,4,Pub,Bus Stop,Restaurant,Deli / Bodega,Convenience Store,Café,Sandwich Place,Bakery,Pharmacy,Fast Food Restaurant
18,51.533331,4,Pub,Bus Stop,Bakery,Restaurant,Café,Convenience Store,Deli / Bodega,Sandwich Place,Metro Station,Garden
22,51.533331,4,Pub,Café,Coffee Shop,Deli / Bodega,Restaurant,Bus Stop,French Restaurant,Gastropub,Sandwich Place,Bakery
